<a href="https://colab.research.google.com/github/emyesme/CalcificationDetection/blob/feature-pm/DeepLearning_MC_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os 

#first put a shortcut in your drive to the image processing folder

RESULTS_DIR = os.path.join('/content',
                        'drive',
                        'MyDrive',
                        'Results')


DATA_DIR = os.path.join('/content',
                        'drive',
                        'MyDrive',
                        'Image Processing and Analysis 2022',
                        'projects',
                        'Calcification Detection',
                        'dataset')


print(os.listdir(RESULTS_DIR))

data_file = os.listdir(RESULTS_DIR)

['X_train.csv', 'test', 'X_train2.csv', 'X_train3.csv', '4.dehazingDC+DoH(0.0005)+GLCM', '5.dehazingDilBlur+DoH+GLCM', 'FROC_calculations_KNN_pip4_5296.csv', 'FROC_calculations_DT_pip4_fn5173_normals57.csv', 'FROC_calculations_DT_pip4_fn294_normals57.csv', 'FROC_calculations_DT_pip6_fn235_normals39.csv', 'FROC_calculations_DT_pip6_fn257_normals37.csv', 'FROC_calculations_DT_pip6_fn203_normals57.csv', 'FROC_calculations_DT_pip4_fn203_normals57.csv', '8.dehazingDil+DoG+GLCM', '6.dehazingMorph+DoH+HaarWaveletGLCM', '10.CLAHE+Dehazing+Dil(3,3)+DoGTweak+GLCM', 'groundTruthStats.gsheet', 'groundTruthStats.csv', 'DehazingDarkChannel_GuidedFilter', 'CLAHE+Dehazing', 'CLAHE+Dehazing+GrayMorph', '10-1.CLAHE+Dehazing+Dil(3,3)+DoGTweak(0.06)+GLCM', 'Pipeline_Prepro10-DoGTweakOf8', 'TestLBP', '13.CLAHE+Dehazing+Dil(3,3)+DoG(10)+LBP+GLCM', '12.CLAHE+Dehazing+GrayMorph+Dil(3,3)+DoGTweak', 'Patches_complete', 'Patches', '10', '12', 'FROC_calculations_SVM-SEL_pip10_fn479_normals57.csv', 'FROC_calculati

In [ ]:
# import the required packages
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.utils.data as data
import copy

In [ ]:
# hyperparameters
batch_size = 32
learning_rate = 0.02
epochs = 30
momentum = 0.1
lr_step_size = 1000   # if < epochs, we are using decaying learning rate
lr_gamma = 0.1
data_augmentation = True
dropout = 0.1
activation = nn.LeakyReLU()

# make visible only one GPU at the time
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # <-- should be the ID of the GPU you want to use

# options
# device = "cuda:0"           # put here "cuda:0" if you want to run on GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
monitor_display = True      # whether to display monitored performance plots
display_first_n = 0         # how many samples/batches are displayed
num_workers = 2             # how many workers (=threads) for fetching data
pretrained = False          # whether to test a pretrained model (to be loaded) or train a new one
display_errors = True       # whether to display errors (only in pretrained mode)

**Load information**

In [ ]:
import roi_cc_project

dataset = roi_cc_project.UnlabeledImageROI(os.path.join(DATA_DIR, 'images', '20586908_6c613a14b80a8591_MG_R_CC_ANON.tif'),os.path.join(DATA_DIR, 'groundtruths', '20586908_6c613a14b80a8591_MG_R_CC_ANON.tif'), (12,12), img_channel=None, preprocessing=None, verbose=True)

...5129587 ROIs extracted in 66 s


In [ ]:
class Convert(object):
    def __call__(self, img):
      return torch.from_numpy(np.transpose(img.astype(np.float32), (2, 0, 1))).float()
        # return torch.unsqueeze(torch.from_numpy(np.array(img)), 0).float()

# define Flatten transform so as to have a 28x28=784 tensor
class Flatten(object):
    def __call__(self, img):
        return img.view(12*12)

# define target transform so as to have a 1-hot tensor
class OneHot(object):
    def __call__(self, label):
        #target = torch.zeros(10, dtype=torch.float)
        #target[label] = 1.0
        return label


transform_train = transforms.Compose(
     [Convert(),
      # transforms.ToTensor(),
     Flatten()])
# transform_test = transforms.Compose(
#     [Convert(),
# #     transforms.Normalize(mean=[mu], std=[std]),
#      Flatten()])



In [ ]:
# check your original data, before applying any transform
# NOTE: we also calculate data mean and standard deviation
print ("\nTrain data are %d, with shape %s" % (len(train_features), train_features.shape))
mu = train_features.float().mean()
std = train_features.float().std()
print ("...with mean %.1f and standard deviation %.1f" % (mu, std))
print ("...with labels %s, %s, %s, ..." % (np.unique(train_labels)[0], np.unique(train_labels)[1]))
# mu_valid = dataset_valid.data.float().mean()
# std_valid = dataset_valid.data.float().std()
# print ("\nValidation data are %d, with shape %s" % (len(dataset_valid), dataset_valid.data.shape))
# print ("...with mean %.1f and standard deviation %.1f" % (mu_valid, std_valid))
# print ("...with labels %s, %s, %s, ..." % (dataset_valid.targets[0], dataset_valid.targets[1], dataset_valid.targets[2]))
# # visual check
# for i in range(0, display_first_n):
#     plt.imshow(dataset_train.data[i], cmap='gray')
#     plt.title('Training Sample %d' % i)
#     plt.show()

NameError: ignored

Train test split

In [ ]:
train_percentage = 0.6
test_percentage = 0.25

train_size = int(len(dataset)*train_percentage)
test_size = int(len(dataset)*test_percentage)

indices = list(range(len(dataset)))
np.random.shuffle(indices)
train_indices, test_indices, val_indices = indices[:train_size], indices[train_size:train_size+test_size], indices[train_size+test_size:]
train_features = data.SubsetRandomSampler(train_indices)
val_features = data.SubsetRandomSampler(val_indices)
test_features = data.SubsetRandomSampler(test_indices)

In [ ]:
dataloader_train = torch.utils.data.DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=True, sampler=train_features)
dataloader_val = torch.utils.data.DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=True, sampler=val_features)
dataloader_test = torch.utils.data.DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=True, sampler=test_features)

dataloader = {'train': dataloader_train, 'eval': dataloader_val, 'test': dataloader_test}
dataset_sizes = {'train': train_size, 'eval': len(dataset)-train_size-test_size,'test': test_size}

Model definition

In [ ]:
# define CNN

###
# Haq, I.U., Ali, H., Yu, W.H., Lei, C., Ali, H., Feature fusion and ensemble learningbased CNN model for mammographic image classification, Journal of King Saud University - Computer and
# Information Sciences (2022), doi: https://doi.org/10.1016/j.jksuci.2022.03.023
###

## leakyrelu used instead of elu
## Adam optimizer with a learning rate of 0.0001, β1 of 0.9, and β2 of 0.99, and a batch size of 24

class CD_CNN(nn.Module):
    def __init__(self):
        super(CD_CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1) #kernel_size=7
        # kernel sizes reduced due to patch size
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1) #kernel_size=7
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2) # stride? padding?
        #https://stackoverflow.com/questions/63971920/why-am-i-getting-calculated-padding-input-size-per-channel-smaller-than-kernel-s
        self.dropout1=nn.Dropout(p=0.2)
        self.bn3 = nn.BatchNorm2d(64)

        #self.dwconv1 = nn.DepthwiseConv2D(in_channels=64, out_channels=128, kernel_size=5, padding=1)
        self.dwconv1 = nn.Conv2d(in_channels=64, out_channels=128, groups= 64, kernel_size=3, padding=1) #kernel_size=5
        #https://discuss.pytorch.org/t/attributeerror-module-torch-nn-has-no-attribute-conv2d/90038
        #https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html        
        self.bn4 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1) #kernel_size=5
        self.bn5 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout2=nn.Dropout(p=0.2)
        self.bn6 = nn.BatchNorm2d(128)

        #self.dwconv2 = nn.DepthwiseConv2D(in_channels=128, out_channels=256, kernel_size=3, padding=1)
       
        # third block removed due to patch size
        # self.dwconv2 = nn.Conv2d(in_channels=128, out_channels=256, groups= 128, kernel_size=3, padding=1)
        # self.bn7 = nn.BatchNorm2d(256)
        # self.conv4 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        # self.bn8 = nn.BatchNorm2d(256)
        # self.conv5 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        # self.bn9 = nn.BatchNorm2d(256)
        # self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        # self.dropout3=nn.Dropout(p=0.2)
        # self.bn10 = nn.BatchNorm2d(256)

        self.fc1 = nn.Linear(128*3*3,512) #128*3*3 is output size of previous stage
        self.fc2 = nn.Linear(512,2) # 2?

        self.out = nn.Sigmoid()

        
        
        self.act = activation

        #self.act = nn.Sigmoid()

        nn.init.xavier_normal_(self.conv1.weight)
        nn.init.xavier_normal_(self.conv2.weight)
        nn.init.xavier_normal_(self.conv3.weight)
       # nn.init.xavier_normal_(self.conv4.weight)
       # nn.init.xavier_normal_(self.conv5.weight)
        nn.init.xavier_normal_(self.dwconv1.weight)
       # nn.init.xavier_normal_(self.dwconv2.weight)


    def forward(self, x):
        
        x = self.act(self.bn1(self.conv1(x)))
        x = self.act(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        x = self.bn3(self.dropout1(x))
        # print(x.size())

        x = self.act(self.bn4(self.dwconv1(x)))
        x = self.act(self.bn5(self.conv3(x)))
        x = self.pool2(x)
        x = self.bn6(self.dropout2(x))
        # print(x.size())

        # x = self.act(self.bn7(self.dwconv2(x)))
        # x = self.act(self.bn8(self.conv4(x))) 
        # x = self.act(self.bn9(self.conv5(x))) 
        # x = self.pool3(x)
        # x = self.bn10(self.dropout3(x))
        # print(x.size())

        x = x.view(-1, 128*3*3)
        x = self.act(self.fc1(x))

        # print(x.size())

        x = self.fc2(x)
        # no activation: "remember in this case to not specify the activation in the last layer when you define the net!"
        # consider softmax/logsoftmax
        
        # print(x.size())
        
        x = self.out(x)

        # print(x.size())

        return x

In [ ]:
class DeepNet(nn.Module):
    def __init__(self):
       super(DeepNet, self).__init__()
       self.fc1 = nn.Linear(144, 100)
       self.fc2 = nn.Linear(100, 100)
       self.fc3 = nn.Linear(100, 10)

       nn.init.xavier_normal_(self.fc1.weight)
       nn.init.xavier_normal_(self.fc2.weight)
       nn.init.xavier_normal_(self.fc3.weight)

       self.bn1 = nn.BatchNorm1d(100)
       self.bn2 = nn.BatchNorm1d(100)

       self.act = activation

       self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        
        x = self.act(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.act(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.fc3(x)

        return x

In [ ]:
class CNN1(nn.Module):
  def __init__(self) -> None:
      super(CNN1, self).__init__()

      # in_channels is number of channels (this is because we have a patch of 1 image)
      # out_channels is number of filters or kernels given out by the convolution
      # stride is a parameter (default is 1)
      self.conv1 =  nn.Conv2d(in_channels=1, out_channels=64, kernel_size=2, padding=1)
      self.bn1 = nn.BatchNorm2d(64)
      self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=2, padding=1)
      self.bn2 = nn.BatchNorm2d(64)
      self.pool1 = nn.MaxPool2d(kernel_size=2)
      self.dropout1=nn.Dropout(p=0.2)
      self.bn3 = nn.BatchNorm2d(64)

      self.flatten = nn.Flatten()

      self.fc1 = nn.Linear(3136,512)
      self.fc2 = nn.Linear(512,2)

      nn.init.xavier_normal_(self.fc1.weight)
      nn.init.xavier_normal_(self.fc2.weight)
      nn.init.xavier_normal_(self.conv1.weight)
      nn.init.xavier_normal_(self.conv2.weight)

  def forward(self, x):

      x = self.conv1(x)
      x = self.bn1(x)
      x = self.conv2(x)
      x = self.conv2(x)
      x = self.pool1(x)
      x = self.dropout1(x)
      x = self.bn3(x)
      x = self.flatten(x)
      x = self.fc1(x)
      x = activation(x)
      x = self.fc2(x)

      return x



In [ ]:
from torchsummary import summary

net = CNN1()
summary(net, (1,12,12))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 13, 13]             320
       BatchNorm2d-2           [-1, 64, 13, 13]             128
            Conv2d-3           [-1, 64, 14, 14]          16,448
            Conv2d-4           [-1, 64, 15, 15]          16,448
         MaxPool2d-5             [-1, 64, 7, 7]               0
           Dropout-6             [-1, 64, 7, 7]               0
       BatchNorm2d-7             [-1, 64, 7, 7]             128
           Flatten-8                 [-1, 3136]               0
Total params: 33,472
Trainable params: 33,472
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.47
Params size (MB): 0.13
Estimated Total Size (MB): 0.59
----------------------------------------------------------------


In [ ]:
dummy_variable = torch.rand(1,1,12,12) #batch size, channel, image size
net(dummy_variable).size()

torch.Size([1, 3136])

In [ ]:
net = CNN1().to(device) #we need to also send the model to the GPU as well

# create loss function
criterion = nn.CrossEntropyLoss() #most used for classification purposes

# create SGD optimizer
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum) #most common optimizer is adam

# create learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=lr_step_size, gamma=lr_gamma)

# experiment ID
experiment_ID = "%s_%s_%s_bs(%d)lr(%.4f_%d_%.1f)m(%.1f)e(%d)act(%s)xavier(yes)da(%s)do(%.1f)BN" % (type(net).__name__, type(criterion).__name__, type(optimizer).__name__,
                batch_size, learning_rate, lr_step_size, lr_gamma, momentum, epochs, type(activation).__name__, data_augmentation, dropout)

In [ ]:
# define train function (1 epoch)
# returns average loss and accuracy
def train(dataset, dataloader):

    # switch to train mode
    net.train()

    # reset performance measures
    loss_sum = 0.0
    correct = 0

    # 1 epoch = 1 complete loop over the dataset
    for batch in dataloader:

        # get data from dataloader. This is the thing that I get from the get_item
        inputs, targets = batch

        # move data to device
        inputs, targets = inputs.to(device, non_blocking=True), targets.to(device, non_blocking=True)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward pass
        outputs = net(inputs)

        # calculate loss
        loss = criterion(outputs, targets)

        # loss gradient backpropagation
        loss.backward() # I calculate the derivatives backwards

        # net parameters update
        optimizer.step() #I use the gradients to update the weights

        # accumulate loss
        loss_sum += loss.item()

        # accumulate correct outputs (for accuracy calculation)
        outputs_max = torch.argmax(outputs, dim=1) #predicted labels
        targets_max = targets #torch.argmax(targets, dim=1)
        correct += outputs_max.eq(targets_max).sum().float() # this is to find out how many predictions were correct

    # step learning rate scheduler
    scheduler.step() #we just need to keep track of the steps since the LR depends on it

    # return average loss and accuracy
    return loss_sum / len(dataloader), 100. * correct / len(dataset)


In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=30):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict()) #It keeps track of the parameters of the model in certain state
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'eval']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloader[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    preds = torch.argmax(outputs, dim=1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            if phase == 'train':
              scheduler.step()

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            

            # deep copy the model
            if phase == 'eval' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
# define test function
# returns predictions
def test(dataset, dataloader):

    # switch to test mode
    net.eval()  

    # initialize predictions
    predictions = torch.zeros(len(dataset), dtype=torch.int64)
    sample_counter = 0

    # do not accumulate gradients (faster)
    with torch.no_grad():

        # test all batches
        for batch in dataloader:

            # get data from dataloader [ignore labels/targets as they are not used in test mode]
            inputs = batch[0]

            # move data to device
            inputs = inputs.to(device, non_blocking=True)

            # forward pass
            outputs = net(inputs)

            # store predictions
            outputs_max = torch.argmax(outputs, dim=1)
            for output in outputs_max:
                predictions[sample_counter] = output
                sample_counter += 1

    return predictions

In [ ]:
model = train_model(net, criterion, optimizer, scheduler, num_epochs=epochs)

Epoch 1/30
----------


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


train Loss: 0.0200 Acc: 0.9971
eval Loss: 0.0189 Acc: 0.9971

Epoch 2/30
----------
train Loss: 0.0191 Acc: 0.9971
eval Loss: 0.0188 Acc: 0.9971

Epoch 3/30
----------


KeyboardInterrupt: ignored

In [ ]:
predictions = test(test_features, dataloader_test)
accuracy = 100. * predictions.eq(test_features.targets).sum().float() / len(test_features)
print ("Accuracy on test set is %.2f" % accuracy)

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    # mean = np.array([0.485, 0.456, 0.406])
    # std = np.array([0.229, 0.224, 0.225])
    # inp = std * inp + mean
    # inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
# inputs, classes = next(iter(loader.rois[0]))

# # Make a grid from batch
# out = torchvision.utils.make_grid(inputs)

# imshow(out)

ValueError: ignored